In [69]:
import pandas as pd
import plotly.graph_objs as go
import plotly.offline as pyo
from plotly.subplots import make_subplots
from scipy.signal import find_peaks
import numpy as np
import os

In [179]:
## helper functions 
def create_line_plot(df, prod_cols, arps_cols , chart_title, html_file_path, show_peaks = False):
    """
    Létrehoz egy vonaldiagramot a megadott DataFrame és oszlopok alapján, majd elmenti HTML formátumban.
    Args:
        df: A DataFrame, amiből az adatokat ábrázoljuk.
        prod_cols: A prod oszlopok listája, amiket meg szeretnénk jeleníteni a diagramon.
        arps_cols: Az aprs oszlopok listája, amiket meg szeretnénk jeleníteni a diagramon.
        arps_cols: Az aprs oszlopok listája, amiket meg szeretnénk jeleníteni a diagramon.
        chart_title: Az ábra címe.
        html_file_path: Az elmentendő HTML fájl elérési útvonala.
        show_peaks: amennyiben megtaláltuk a csúcsokat azt ki is rajzolhatjuk.
    Returns:
    """

    fig = make_subplots(specs=[[{"secondary_y": True}]])

    # Az egyes adatsorok (trace-ek) hozzáadása az ábrához
    fig.add_trace(go.Scatter(x=df.index, y=df['BOE'], mode='lines', name='BOE'),secondary_y=False)
    
    if show_peaks==True:
        fig.add_trace(go.Scatter(x=df.loc[df['peaks']>0,:].index, y=df.loc[df['peaks']>0,'peaks'], mode='markers', name='peaks'),secondary_y=True)
       
    # prod oszlopok kumulált ábrázolása
    for i,col in enumerate(prod_cols):
        first_valid_index = df[arps_cols[i]].first_valid_index()
        fig.add_trace(go.Scatter(x=df.loc[first_valid_index:,:].index, y=df.loc[first_valid_index:,prod_cols[:i+1]].sum(axis=1), mode='lines', name=col),secondary_y=False)
    
    # arps oszlopok kumulált ábrázolása   
    for i,col in enumerate(arps_cols):
        first_valid_index = df[arps_cols[i]].first_valid_index()
        fig.add_trace( go.Scatter(x=df.loc[first_valid_index:,:].index, y=df.loc[first_valid_index:,arps_cols[:i+1]].sum(axis=1), mode='lines', name=col),secondary_y=False)
    
    # Ábra beállításai
    layout = go.Layout(title=chart_title)   
    
    # HTML fájlba mentés
    pyo.plot(fig, filename=html_file_path, auto_open=False)
    
def find_peaks_points(df ):    
    
    """
    Csúcsok keresése egy adatsorban (df) csúcsok alapján.
    
    Args:
      df: Pandas adatsor, amelynek 'BOE' nevű oszlopában az értékek szerepelnek.
    
    Returns:
      df: A bemeneti adatsor kibővítve a következő oszlopokkal:
        * `peaks`: Bináris jelölés a csúcsokra (1 - csúcs, 0 - nem csúcs).
    """
    
    peaks, _ = find_peaks(df['BOE'], height=100, threshold = 0.5, distance=17, prominence=None, width=3)    
    # Új oszlopok inicializálása
    df['peaks'] = 0  # Bináris jelölés a csúcsokra
    
    # Végigiterálás a csúcsokon és az eltérések számítása
    for peak in peaks:
        df.at[df.index[peak], 'peaks'] = 1  # Jelöljük a csúcsot  
    
    return df

In [104]:
params = {
          'file_path':'dataset/',
          'save_html_path':'plots/'
         }

In [105]:
# kesessük meg az összes adat nevét
file_list = [file for file in os.listdir(params['file_path']) if 'pkl' in file]

In [106]:
#olvassunk be egy adatot
i = 2851
file = file_list[i]
df = pd.read_pickle(params['file_path'] + file)

In [107]:
# keressük meg a plot oszlopait
prod_cols = [col for col in df.columns if (('prod' in col) and ('_' not in col))]
arps_cols = [col for col in df.columns if (('arps' in col) and ('_' not in col))]
used_cols = ['BOE']

In [108]:
# keszitsünk egy plot-ot és mentsük le azt
chart_title = file.split('.')[0]
html_file_path = params['save_html_path'] + chart_title + '.html'
create_line_plot(df,prod_cols, arps_cols , chart_title, html_file_path)

In [180]:
# keressük meg a csúcsokat
df = find_peaks_points(df)
used_cols = ['BOE','peaks']

In [181]:
# keszitsünk egy plot-ot és mentsük le azt
html_file_path = params['save_html_path'] + chart_title + '_with_peaks' +  '.html'
create_line_plot(df,prod_cols, arps_cols , chart_title, html_file_path, True)